In [50]:
import random
from typing import Tuple, List

import gif
import plotly.graph_objects as go
import numpy as np


# COLORS = [("#155b92", "#15925e"),
#           ("#9b76bc", "#618da7"),
#           ("#ffba08", "#ff8c61"),
#           ("#ffba08", "#43aa8b"),
#           ("#ff928b", "#b392ac"),
#           ("#00d59e", "#ab63fa")]


COLORS = [("#4c5b5c", "#ff715b", "#f9cb40"),
          ("#512d38", "#b27092", "#f4bfdb"),
          ("#b24c63", "#5438dc", "#357ded"),
          ("#05a8aa", "#b8d5b8", "#d7b49e"),
          ("#cce8cc", "#f6efee", "#e2b6cf"),
          ("#cce8cc", "#f6efee", "#e2b6cf")]

# cce8cc-f6efee-e2b6cf
# ffbf00-e83f6f-2274a5


@gif.frame
def animation_frame(fig: go.Figure) -> go.Figure:
    return fig


def make_plot(z: np.ndarray, colors: Tuple[str]) -> go.Figure:
#     height, width = z.shape
    height, width = 50, 50
    fig = go.Figure(go.Heatmap(z=z[::-1], colorscale=colors))
    fig.update_traces(showscale=False)
    fig.update_layout(width=10*width,
                      height=10*height,
                      xaxis_visible=False,
                      yaxis_visible=False,
                      margin=dict(t=0, b=0, l=0, r=0))
    return fig


def make_animation(arrays: List[np.ndarray], filename: str):
    print("animating...")
    colors = random.sample(random.choice(COLORS), k=3)
    frames = [animation_frame(make_plot(arr, colors)) for arr in arrays]
    filename += f"_{len(frames)}_frames.gif"
    gif.save(frames, filename, duration=25)
    print("...done!")


class Maze:
    def __init__(self, x: int, y: int, rooms=False) -> None:
        self.shape = (x + 1, y + 1)
        self.maze = np.zeros(self.shape, dtype="uint8")
        self.grid = [(i, j) for i in range(1, x+1, 2) for j in range(1, y+1, 2)]
        self.path = [random.choice(self.grid)]
        self.position = self.path[0]
        self.grid.remove(self.position)
        self.history = []

        while self.grid:
            for probe_link in self.moves:
                probe, link = probe_link
                if probe in self.grid:
                    self.maze[tuple(zip(probe, link))] = 1
                    self.grid.remove(probe)
                    self.path.append(probe)
                    self.position = probe
                    self.history.append(self.maze.copy())
                    break
            else:
                last_position = self.path.index(self.position)
                self.position = self.path[max(last_position - 1, 1)]
                maze_copy = self.maze.copy()
                maze_copy[self.position] = 2
                self.history.extend([maze_copy]*3)

        self.history.extend([self.history[-1]]*50)

    @property
    def moves(self):
        x, y = self.position
        nsew = [[(x + 2, y), (x + 1, y)],
                [(x, y + 2), (x, y + 1)],
                [(x - 2, y), (x - 1, y)],
                [(x, y - 2), (x, y - 1)]]
        return random.sample(nsew, k=4)

In [8]:
make_animation(m.history, "maze_animation")

NameError: name 'm' is not defined

In [3]:
import numpy as np

size = 15
x = np.zeros((size, size))
x[(4, 5, 5, 6), (5, 4, 6, 5)] = 1
x[(9, 9, 9, 10, 10, 11, 11), (8, 9, 10, 8, 10, 8, 10)] = 1

In [4]:
from scipy import ndimage

In [51]:
z = x + ndimage.binary_dilation(x).astype(int)
make_plot(x + ndimage.morphological_gradient(x, size=(3, 3)), COLORS[0])

In [67]:
dilated = ndimage.binary_dilation(x).astype(int)
eroded = ndimage.binary_erosion(dilated)
borders = dilated-eroded
make_plot(x + ndimage.convolve(x, np.eye(3)) + ndimage.convolve(x, np.fliplr(np.eye(3))), COLORS[0])
# 2 * borders + x

In [58]:
np.fliplr(np.eye(3))

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.]])